在运行dw相关表之前，首先运行下面部分以创建中间表

In [1]:
import pandas as pd
from pyspark import SparkContext,SQLContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

In [2]:
spark=SparkSession \
        .builder \
        .config("spark.eventLog.enabled", "false") \
        .config("spark.executor.memory", "4g")\
        .config("spark.driver.memory", "8g")\
        .config("spark.cores.max", "10")\
        .config("spark.task.maxFailures", "1000")\
        .config("spark.default.parallelism", "500")\
        .config("spark.sql.shuffle.partitions",100)\
        .appName('renhang_etl') \
        .master('yarn')\
        .getOrCreate()

In [3]:
#创建DW.dw_baseinfo_detail
table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRMESSAGEHEADER") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table1.createOrReplaceTempView("ICRMESSAGEHEADER")

table2 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRCREDITCUE") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table2.createOrReplaceTempView("ICRCREDITCUE")

table3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRLOANCARDINFO") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table3.createOrReplaceTempView("ICRLOANCARDINFO")

table4 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRCREDITCUE") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table4.createOrReplaceTempView("ICRCREDITCUE")

table5 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRLATEST2YEAROVERDUECARD") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table5.createOrReplaceTempView("ICRLATEST2YEAROVERDUECARD")

table6 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRLATEST5YEAROVERDUEDETAIL") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table6.createOrReplaceTempView("ICRLATEST5YEAROVERDUEDETAIL")

table7 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRLATEST2YEAROVERDUE") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table7.createOrReplaceTempView("ICRLATEST2YEAROVERDUE")

table8 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRLOANINFO") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table8.createOrReplaceTempView("ICRLOANINFO")

table9 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRUNDESTORYLOANCARD") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table9.createOrReplaceTempView("ICRUNDESTORYLOANCARD")

table10 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRUNDESTORYSTANDARDLOANCARD") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table10.createOrReplaceTempView("ICRUNDESTORYSTANDARDLOANCARD")

table11 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICRUNPAIDLOAN") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table11.createOrReplaceTempView("ICRUNPAIDLOAN")


table12 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "ICR_PROD.ICROVERDUESUMMARY") \
    .option("user", "etl") \
    .option("password", "%TGRDJh3Hg2e4f") \
    .load()
table12.createOrReplaceTempView("ICROVERDUESUMMARY")

In [4]:
spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='准贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_standcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_creditcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		ordue.LASTMONTHS,ordue.AMOUNT,loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'  and BIZTYPE='贷记卡'
""").createOrReplaceTempView("temp_creditcard_overdue")


spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		date_format(concat(month,'.01'),'%Y-%m'),
        gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_overdue
""").createOrReplaceTempView("temp_creditcard_5yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,
		STATE,
		FINANCEORG
from ICRLATEST2YEAROVERDUE ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_loan_2yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
		month(date_format(head.QUERYTIME,'%Y-%m-%d')) - month(date_format(concat(ordue.month,'.01'),'%Y-%m-%d')) as gap_months,
		ordue.LASTMONTHS,
		ordue.AMOUNT,
		loanfo.GUARANTEETYPE,
		STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'
""").createOrReplaceTempView("temp_loan_overdue")




spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
        MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		date_format(concat(month,'.01'),'%Y-%m'),gap_months,LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_overdue
""").createOrReplaceTempView("temp_loan_5yearoverdue")


中间表已运行完成，spark sql 创建所需要的表

In [5]:
spark.sql("""

select 
		ovdue.reportno,
		case 
			when loan.all_loan_account is null then -9999999
			when loan.all_loan_account is not null and ovdue1.overdue_loan_account_num is null then 0
			else ovdue1.overdue_loan_account_num 
		end as overdue_loan_account_num,
		case 
			when loan.all_loan_account=0 or loan.all_loan_account is null then -9999998 
			when (loan.all_loan_account!=0 and loan.all_loan_account is not null) and ovdue1.overdue_loan_account_num is null then 0
			else round(ovdue1.overdue_loan_account_num/loan.all_loan_account,5)  
		end as overdue_loan_account_percent,
		case 
			when loan.all_loan_account is null then -9999999
			when loan.all_loan_account is not null and ovdue1.overdue_loan_account_num is null then 0
			else loanbal.overdue_loan_account_balance 
		end as overdue_loan_account_balance,
		case 
			when loan_sum=0 or loan.loan_sum is null then -9999998
			when loan_sum!=0 and loan.loan_sum is not null and ovdue1.overdue_loan_account_num is null then 0
			else round(loanbal.overdue_loan_account_balance/loan_sum,5) 
		end as overdue_loan_balance_percent,
		nvl(sumy.highest_loan_overdue_permon,-9999999) as highest_loan_overdue_permon,
		nvl(sumy.highest_ccard_overdue_permon,-9999999) as highest_ccard_overdue_permon,
		nvl(sumy.highest_scard_overdue_permon,-9999999) as highest_scard_overdue_permon,
		nvl(sumy.max_loan_duration,-9999999) as max_loan_duration,
		nvl(sumy.max_ccard_duration,-9999999) as max_ccard_duration,
		nvl(sumy.max_scard_duration,-9999999) as max_scard_duration,
		case 
			when loan.all_loan_account is null then -9999999
			when loan.all_loan_account is not null and ovdue1.reportno is null then nvl(sumy.loan_longest_month,0)
			else lmon.loan_overdue_months 
		end as loan_overdue_months,
		case 
			when ccard.reportno is null then -9999999
			when ccard.reportno is not null and cmon.reportno is null then nvl(sumy.ccard_longest_month,0)
			else cmon.ccard_overdue_months 
		end as ccard_overdue_months,
		case 
			when scard.reportno is null then -9999999
			when scard.reportno is not null and smon.reportno is null then nvl(sumy.scard_longest_month,0)
			else smon.scard_overdue_months 
		end as scard_overdue_months,	
		case 
			when loan.all_loan_account is null then -9999999
			when loanfo.reportno is null and loan.all_loan_account is not null then 0
			else loanfo.curoverdue_loan_num 
		end as curoverdue_loan_num,
		case 
			when loan.all_loan_account is null then -9999999
			when loanfo.reportno is null and loan.all_loan_account is not null then 0
			else loanfo.curoverdue_loan_amount 
		end as curoverdue_loan_amount,
		case 
			when loan.all_loan_account is null then -9999999
			when loanfo.reportno is null and loan.all_loan_account is not null then 0
			else loanfo.curoverdue_longest_months 
		end as curoverdue_longest_months,
		case 
			when loan.all_loan_account is null then -9999999
			when loanfo.reportno is null and loan.all_loan_account is not null then 0
			else loanfo.curoverdue_org_num 
		end as curoverdue_org_num,	
		case 
			when ccard.reportno is null then -9999999
			when ccard.reportno is not null and cdtod.reportno is null then 0
			else cdtod.curoverdue_creditcard_account_num 
		end as curoverdue_creditcard_account_num,
		case 
			when ccard.reportno is null then -9999999
			when ccard.reportno is not null and cdtod.reportno is null then 0
			else cdtod.curoverdue_creditcard_org_num 
		end as curoverdue_creditcard_org_num,
		case 
			when ccard.reportno is null then -9999999
			when ccard.reportno is not null and cdtod.reportno is null then 0
			else cdtod.curoverdue_creditcard_amount 
		end as curoverdue_creditcard_amount,	
		case 
			when ccard.reportno is null then -9999999
			when ccard.reportno is not null and cdtod.reportno is null then 0
			else cdtod.curoverdue_creditcard_max 
		end as curoverdue_creditcard_max,
	now() as create_time,now() as update_time,now() as etl_date

from 
(
	select 
		distinct reportno 
	from ICRMESSAGEHEADER
) ovdue
left join 
(-- 发生过逾期的贷款账户笔数
	select 
		a.reportno,
		count(distinct a.account) as overdue_loan_account_num
	from temp_loan_5yearoverdue a
	group by a.reportno
) ovdue1 on ovdue.reportno=ovdue1.reportno
-- 贷款总笔数、贷款总余额
left join 
(
	select 
		reportno,
		count(*) as all_loan_account,
		sum(nvl(BALANCE,0)) as loan_sum
	from ICRLOANINFO
	group by reportno
) loan on ovdue.reportno=loan.reportno
left join 
(	-- 发生过逾期的贷款账户余额
	select 
		a.reportno,
		sum(nvl(b.BALANCE,0)) as overdue_loan_account_balance
	from (
		select 
			distinct reportno,
			account
		from temp_loan_5yearoverdue 
		)a
	left join ICRLOANINFO b on a.reportno=b.reportno and a.account=b.account
	group by a.reportno
)loanbal on ovdue.reportno=loanbal.reportno
left JOIN
(-- 贷款、贷记卡、准贷记卡单月最高逾期金额/最长逾期月数
select 
		reportno,
		HIGHESTOVERDUEAMOUNTPERMON as highest_loan_overdue_permon,
		HIGHESTOVERDUEAMOUNTPERMON2 as highest_ccard_overdue_permon,
		HIGHESTOVERDUEAMOUNTPERMON3 as highest_scard_overdue_permon,
		MAXDURATION as max_loan_duration,
		MAXDURATION2 as max_ccard_duration,
		MAXDURATION3 as max_scard_duration,
		MONTHS3 as scard_longest_month,
		MONTHS2 as ccard_longest_month,
		MONTHS as loan_longest_month
from ICROVERDUESUMMARY
)sumy on ovdue.reportno=sumy.reportno
-- 贷款发生过逾期的月份数
left join
(	
	select 
		reportno,
		count(distinct month) as loan_overdue_months
	from temp_loan_5yearoverdue
	group by reportno
)lmon on ovdue.reportno=lmon.reportno
-- 贷记卡发生过逾期的月份数
left join
(	select 
		reportno,
		count(distinct month) as ccard_overdue_months
	from temp_creditcard_5yearoverdue
	group by reportno
)cmon on ovdue.reportno=cmon.reportno
-- 准贷记卡发生过逾期的月份数
left join
(	select 
		reportno,
		count(distinct month) as scard_overdue_months
	from temp_standcard_2yearoverdue
	group by reportno
)smon on ovdue.reportno=smon.reportno

left JOIN
(-- 当前处于逾期的贷款
select 
	reportno,
	count(*) as curoverdue_loan_num,
	sum(nvl(CURROVERDUEAMOUNT,0)) as curoverdue_loan_amount,
	count(distinct FINANCEORG) as curoverdue_org_num,
	max(case when substr(LATEST24STATE,-1) in ('G','Z','D') then 8 
		else substr(LATEST24STATE,-1) end) as curoverdue_longest_months
from ICRLOANINFO 
where LATEST24STATE is not null and substr(LATEST24STATE,-1) not in('/', '#', 'N', '*','C')
group by reportno
) loanfo on ovdue.reportno=loanfo.reportno
/*
-- 当前贷款最长逾期期数
left join
(select reportno,max(LASTMONTHS) as curoverdue_longest_months 
from DW.temp_loan_5yearoverdue
group by reportno
)loan1 on ovdue.reportno=loan1.reportno
*/
-- 当前处于逾期的贷记卡
left join 
(	
	select 
		reportno,
		count(distinct  ACCOUNT) as curoverdue_creditcard_account_num,
		count(distinct FINANCEORG) as curoverdue_creditcard_org_num,
		sum(nvl(CURROVERDUEAMOUNT,0)) as curoverdue_creditcard_amount,
		max(substr(LATEST24STATE,-1)) as curoverdue_creditcard_max
	
	from ICRLOANCARDINFO
	where LATEST24STATE is not null and substr(LATEST24STATE,-1) not in('/', '#', 'N', '*','C')
		and biztype='贷记卡'
	group by reportno
) cdtod on ovdue.reportno=cdtod.reportno
left join 
(	select reportno,count(*)  as ccard_num 
	from ICRLOANCARDINFO
	where biztype='贷记卡'
	group by reportno
)ccard on ovdue.reportno=ccard.reportno
left join 
(	select reportno,count(*)  as scard_num 
	from ICRLOANCARDINFO
	where biztype='准贷记卡'
	group by reportno
)scard on ovdue.reportno=scard.reportno



""").createOrReplaceTempView("dw_overdue_account")



In [6]:
table_name = "dw_overdue_account"

spark.sql("drop table if EXISTS renhang_user_profile.%s "%table_name)
spark.sql("create table renhang_user_profile.%s as select * from %s"%(table_name,table_name))

DataFrame[]